In [ ]:
def random_tt_binary_fixed(seed=None):
    """
    Create a random TT with 7 binary modes and ranks [1,2,4,8,8,4,2,1].

    Parameters
    ----------
    seed : int or None
        Random seed for reproducibility.

    Returns
    -------
    cores : list of ndarray
        List of 7 cores, shapes as described.
    """
    if seed is not None:
        np.random.seed(seed)
    dims  = [2] * 7
    ranks = [1, 2, 4, 8, 8, 4, 2, 1]
    cores = []
    for k in range(7):
        rL, n, rR = ranks[k], dims[k], ranks[k+1]
        G = np.random.randn(rL, n, rR)
        cores.append(G)
    return cores

def tt_to_tensor(cores):
    """
    Reconstruct the full tensor from its TT cores.
    """
    # start with first core, squeeze r0=1
    A = np.squeeze(cores[0], axis=0)   # shape (2, r1)
    for G in cores[1:]:
        A = np.tensordot(A, G, axes=([-1],[0]))
    # squeeze final r7=1
    return np.squeeze(A, axis=-1)      # final shape (2,2,...,2), 7 modes

if __name__ == "__main__":
    cores = random_tt_binary_fixed(seed=2025)
    print("Core shapes:", [c.shape for c in cores])
    A = tt_to_tensor(cores)
    print("Reconstructed tensor shape:", A.shape)
    # Now A[i1,i2,...,i7] exists for each ik∈{0,1}

print(A)

In [ ]:
W = []
nk = np.array([1,2,4,8,8,4,2])

for k in range(len(cores)):
        
        j, i, n_k = cores[k].shape
        #print(cores[k].shape)
        cores[k] = np.reshape(cores[k], (j * i, n_k)) 
        #print(cores[k].shape)       
        U, S, V = np.linalg.svd(cores[k])
        R = np.diag(S) @ V
        if k != len(cores)-1:
            cores[k+1] = np.einsum('j r,r i k->j i k', R, cores[k+1])
        W.append(U)


for k, U in enumerate(W):
    print(U.shape)
    start = k + 1
    mn    = min(start, np.log2(nk[k])) 
    diff  = int(start - mn)
    print(f"This unitary acts from qubit {int(diff)} to qubit {start}")

    #for i in range(0,diff+1):
    #     U = np.kron(np.eye(2),U)

    #for j in range(start+1,len(W)):
    #     U = np.kron(U,np.eye(2))

    print(U.shape)
